In [ ]:
# !pip install uvicorn
# !pip install fastapi
# !pip install python-multipart

In [1]:
import io
import uvicorn
import numpy as np
import nest_asyncio
from enum import Enum
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import StreamingResponse
import os

In [2]:
import pickle
import joblib
import librosa

def extract(file):
    source, sr = librosa.load(file, res_type="kaiser_fast")
    stft = np.abs(librosa.stft(source))

    mfcc = librosa.feature.mfcc(y=source, sr=sr, n_mfcc=13)
    chroma = librosa.feature.chroma_stft(S=stft, sr=sr)
    mel = librosa.feature.melspectrogram(y=source, sr=sr)
    zrc = librosa.feature.zero_crossing_rate(source)

    mfcc = np.mean(mfcc, axis=1)
    chroma = np.mean(chroma, axis=1)
    mel = np.mean(mel, axis=1)
    zcr = np.mean(zrc)

    return mfcc, chroma, mel, zcr

def scale(features, name):
    scaler = joblib.load(f"weights/scalers/scaler_{name}.save")

    return scaler.transform(features.reshape(1, -1))

def predict(file):

    mfcc, chroma, mel, zcr = extract(file)
    
    zcr = scale(zcr, "zcr").reshape(-1)
    mfcc = scale(mfcc, "mfcc").reshape(-1)
    chroma = scale(chroma, "chroma").reshape(-1)
    mel = scale(mel, "mel").reshape(-1)

    x = np.concatenate([zcr, mfcc, chroma, mel], axis=0).reshape(1, -1)

    res = 0

    model_list = os.listdir(f"weights/models/")

    for name in model_list:
        model = pickle.load(open(f"weights/models/" + name, encoding="utf8"))

        res += model.predict(x)

    return (res /len(model_list))[0]

In [ ]:
app = FastAPI(title='Deploying with FastAPI')


# By using @app.get("/") you are allowing the GET method to work for the / endpoint.

@app.get("/")
def home():
    return "Congratulations! Your API is working as expected. Author: Tung Khong Manh. Now head over to " \
           "http://localhost:8000/docs. "


@app.post("/predict")
async def prediction(fileUpload: UploadFile = File(...)):
    filename = fileUpload.filename
    fileExtension = filename.split(".")[-1] in ("wav")
    if not fileExtension:
        raise HTTPException(status_code=415, detail="Unsupported file provided.")
    pred = predict(fileUpload.file)
    return {"result": 0.7}
   
# Allows the server to be run in this interactive environment
nest_asyncio.apply()

# Host depends on the setup you selected (docker or virtual env)
host = "0.0.0.0" if os.getenv("DOCKER-SETUP") else "127.0.0.1"

# Spin up the server!
uvicorn.run(app, host=host, port=8000)

INFO:     Started server process [4680]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:51609 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:51609 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:58564 - "POST /predict HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "D:\anaconda3\envs\AICOVIDVN__KO-Hackathon-II-2021\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 373, in run_asgi
    result = await app(self.scope, self.receive, self.send)
  File "D:\anaconda3\envs\AICOVIDVN__KO-Hackathon-II-2021\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 75, in __call__
    return await self.app(scope, receive, send)
  File "D:\anaconda3\envs\AICOVIDVN__KO-Hackathon-II-2021\lib\site-packages\fastapi\applications.py", line 208, in __call__
    await super().__call__(scope, receive, send)
  File "D:\anaconda3\envs\AICOVIDVN__KO-Hackathon-II-2021\lib\site-packages\starlette\applications.py", line 112, in __call__
    await self.middleware_stack(scope, receive, send)
  File "D:\anaconda3\envs\AICOVIDVN__KO-Hackathon-II-2021\lib\site-packages\starlette\middleware\errors.py", line 181, in __call__
    raise exc from None
  File "D:\anaconda3\envs\AICOVIDVN__KO

In [ ]:
# Allows the server to be run in this interactive environment
nest_asyncio.apply()

# Host depends on the setup you selected (docker or virtual env)
host = "0.0.0.0" if os.getenv("DOCKER-SETUP") else "127.0.0.1"

# Spin up the server!    
uvicorn.run(app, host=host, port=8000)